In [49]:
import nltk 
nltk.download("wordnet")
nltk.download("omw-1.4")
import numpy as np
from nltk.corpus import wordnet as wn
import pandas as pd

[nltk_data] Downloading package wordnet to C:\Users\apl-
[nltk_data]     vr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\apl-
[nltk_data]     vr\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [38]:
def getSynset(term):
    synsets = []
    synsets.extend(wn.synsets(term))
    if len(term) > 1 and len(synsets) == 0: 
        for subterm in term.split():
            synsets.extend(wn.synsets(subterm))
    return np.unique(synsets).tolist()
def computeSynsetSimilarity(synset1, synset2):
    return synset1.path_similarity(synset2)
def computeSynsetsSimilarity(synsets1, synsets2):
    maxSimilarity = 0
    for synset1 in synsets1:
        for synset2 in synsets2: 
            similarity = computeSynsetSimilarity(synset1, synset2)
            if similarity == None:
                continue
            if similarity > maxSimilarity:
                maxSimilarity = similarity
    return maxSimilarity
def getRelated(term):
    related = [] 
    for synset in getSynset(term):
        for relatedSynset in synset.hypernyms():
            for lemmaName in relatedSynset.lemma_names():
                related.append(lemmaName.replace("_", " "))
        for relatedSynset in synset.instance_hypernyms():
            for lemmaName in relatedSynset.lemma_names():
                related.append(lemmaName.replace("_", " "))
        for relatedSynset in synset.hyponyms():
            for lemmaName in relatedSynset.lemma_names():
                related.append(lemmaName.replace("_", " "))
        for relatedSynset in synset.instance_hyponyms():
            for lemmaName in relatedSynset.lemma_names():
                related.append(lemmaName.replace("_", " "))
        for relatedSynset in synset.member_holonyms():
            for lemmaName in relatedSynset.lemma_names():
                related.append(lemmaName.replace("_", " "))
        for relatedSynset in synset.substance_holonyms():
            for lemmaName in relatedSynset.lemma_names():
                related.append(lemmaName.replace("_", " "))
        for relatedSynset in synset.part_holonyms():
            for lemmaName in relatedSynset.lemma_names():
                related.append(lemmaName.replace("_", " "))
        for relatedSynset in synset.member_meronyms():
            for lemmaName in relatedSynset.lemma_names():
                related.append(lemmaName.replace("_", " "))
        for relatedSynset in synset.substance_meronyms():
            for lemmaName in relatedSynset.lemma_names():
                related.append(lemmaName.replace("_", " "))
        for relatedSynset in synset.part_meronyms():
            for lemmaName in relatedSynset.lemma_names():
                related.append(lemmaName.replace("_", " "))
        for relatedSynset in synset.attributes():
            for lemmaName in relatedSynset.lemma_names():
                related.append(lemmaName.replace("_", " "))
        for relatedSynset in synset.entailments():
            for lemmaName in relatedSynset.lemma_names():
                related.append(lemmaName.replace("_", " "))
        for relatedSynset in synset.causes():
            for lemmaName in relatedSynset.lemma_names():
                related.append(lemmaName.replace("_", " "))
        for relatedSynset in synset.also_sees():
            for lemmaName in relatedSynset.lemma_names():
                related.append(lemmaName.replace("_", " "))
        for relatedSynset in synset.verb_groups():
            for lemmaName in relatedSynset.lemma_names():
                related.append(lemmaName.replace("_", " "))
        for relatedSynset in synset.similar_tos():
            for lemmaName in relatedSynset.lemma_names():
                related.append(lemmaName.replace("_", " "))
    return np.unique(related).tolist()
def parseAssociations(fname):
    elements = []
    for e in open(fname).read().split("\n"):
        info = e.split(",")
        element = info[0]
        associations = []
        synsets = []
        for association in info[1:]:
            if len(association) <= 0:
                continue
            associations.append(association)
            synsets = synsets + getSynset(association)
        elements.append({
            "element": element,
            "associations": associations,
            "synsets": synsets
        })
    return elements

In [55]:
elementAnchors = parseAssociations("AssociationsElementsAnchor.csv")
elementAvoids = parseAssociations("AssociationsElementsAvoid.csv")
objAssociations = parseAssociations("AssociationsObjects.csv")

In [71]:
associations = {}
for element in elementAnchors: 
    associations[element["element"]] = {}
    for obj in objAssociations:
        associations[element["element"]][obj["element"]] = computeSynsetsSimilarity(element["synsets"], obj["synsets"])
pd.DataFrame(associations).to_csv("elementObjAnchor.csv")

In [70]:
associations = {}
for element in elementAvoids: 
    associations[element["element"]] = {}
    for obj in objAssociations:
        associations[element["element"]][obj["element"]] = computeSynsetsSimilarity(element["synsets"], obj["synsets"])
pd.DataFrame(associations).to_csv("elementObjAvoid.csv")

In [3]:
import pandas as pd
df_avoid = pd.read_csv("elementObjAvoid.csv",index_col=0)
df_avoid["MusicPlayer"]["Computer"]

0.1428571428571428